## Creating the Index

In [1]:
print("teste")

teste


In [2]:
!pip install llama_index datasets llama-index-llms-huggingface llama-index-retrievers-bm25


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [3]:
import datasets
from llama_index.core.schema import Document

guest_dataset = datasets.load_dataset("agents-course/unit3-invitees", split="train")

docs = [
    Document(
        text="\n".join([
          f"Name: {guest_dataset['name'][i]}",
          f"Relation: {guest_dataset['relation'][i]}",
          f"Description: {guest_dataset['description'][i]}",
          f"Email: {guest_dataset['email'][i]}"
    ]),
        metadata={"name": guest_dataset['name'][i]}
    )
     for i in range(len(guest_dataset))
]

/home/william/repos/agentic-rag/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
docs[0]

Document(id_='caba66d7-75f2-4fe1-b10b-bc17f6a09978', embedding=None, metadata={'name': 'Ada Lovelace'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text="Name: Ada Lovelace\nRelation: best friend\nDescription: Lady Ada Lovelace is my best friend. She is an esteemed mathematician and friend. She is renowned for her pioneering work in mathematics and computing, often celebrated as the first computer programmer due to her work on Charles Babbage's Analytical Engine.\nEmail: ada.lovelace@example.com", path=None, url=None, mimetype=None), image_resource=None, audio_resource=None, video_resource=None, text_template='{metadata_str}\n\n{content}')

## Building the retriever

In [5]:
from llama_index.core.tools import FunctionTool
from llama_index.retrievers.bm25 import BM25Retriever

bm25_retriever = BM25Retriever.from_defaults(nodes=docs)

def get_guest_info_retriever(query: str) -> str:
    """Retrieves detailed information about gala guests based on their name or relation."""
    results = bm25_retriever.retrieve(query)
    if results:
        return "\n\n".join([doc.text for doc in results[:3]])
    else:
        return "No matching guest information found."

# Initialize the tool
guest_info_tool = FunctionTool.from_defaults(get_guest_info_retriever)

## Building the Agent

In [6]:
!pip install llama-index-llms-huggingface-api


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [7]:
!pip install llama-index-llms-ollama


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [8]:
from llama_index.core.agent.workflow import AgentWorkflow
from llama_index.llms.ollama import Ollama

# Initialize the Hugging Face model
llm = Ollama(
        model="llama3.2",
        temperature=0.1,
        request_timeout=120.0
    )

alfred = AgentWorkflow.from_tools_or_functions(
    [guest_info_tool],
    llm=llm
)

In [9]:
# Example query Alfred might receive during the gala
response = await alfred.run("Tell me about our guest named 'Lady Ada Lovelace'.")

print("🎩 Alfred's Response:")
print(response)

🎩 Alfred's Response:
Lady Ada Lovelace was a mathematician and computer scientist who is widely recognized as the world's first computer programmer. She is best known for her work on Charles Babbage's proposed mechanical general-purpose computer, the Analytical Engine.

In 1843, Ada Lovelace wrote a set of notes on the Analytical Engine, which included what is considered to be the first algorithm intended to be processed by a machine. Her notes were written in response to an article about the engine by Italian mathematician Luigi Menabrea, and they provided a detailed analysis of how the machine could be used to perform calculations and manipulate data.

Ada Lovelace's work on the Analytical Engine was well ahead of its time, and it laid the foundation for modern computer science. She is often credited with recognizing the potential of machines to go beyond mere calculation and perform any task that could be expressed in a series of steps.

Today, Ada Lovelace is celebrated as a pionee

## Giving Alfred access to the web

In [10]:
!pip install llama-index-tools-duckduckgo


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [11]:
from llama_index.tools.duckduckgo import DuckDuckGoSearchToolSpec
from llama_index.core.tools import FunctionTool

tool_spec = DuckDuckGoSearchToolSpec()
search_tool = FunctionTool.from_defaults(tool_spec.duckduckgo_full_search)

response = search_tool("Who's the current President of France?")
print(f"🔍 Search Tool Response: {response.raw_output[-1]['body']}")


🔍 Search Tool Response: Emmanuel Macron, born on December 21, 1977, is the current president of the French Republic.In office since May 14, 2017, he was re-elected on April 24, 2022 with 58.55% of the votes against his competitor Marine Le Pen. With a background in higher public service and investment banking, Macron approaches his mandate with a desire for change and innovation in various fields, including economics ...


## Custom weather search tool

In [12]:
import random
from llama_index.core.tools import FunctionTool

def get_weather_info(location: str) -> str:
    """Fetches dummy weather information for a given location."""
    # Dummy weather data
    weather_conditions = [
        {"condition": "Rainy", "temp_c": 15},
        {"condition": "Clear", "temp_c": 25},
        {"condition": "Windy", "temp_c": 20}
    ]
    # Randomly select a weather condition
    data = random.choice(weather_conditions)
    return f"Weather in {location}: {data['condition']}, {data['temp_c']}°C"

# Initialize the tool
weather_info_tool = FunctionTool.from_defaults(get_weather_info)

## AI Builders

In [13]:
from llama_index.core.tools import FunctionTool
from huggingface_hub import list_models

def get_hub_stats(author: str) -> str:
    """Fetches the most downloaded model from a specific author on the Hugging Face Hub."""
    try:
        # List models from the specified author, sorted by downloads
        models = list(list_models(author=author, sort="downloads", direction=-1, limit=1))

        if models:
            model = models[0]
            return f"The most downloaded model by {author} is {model.id} with {model.downloads:,} downloads."
        else:
            return f"No models found for author {author}."
    except Exception as e:
        return f"Error fetching models for {author}: {str(e)}"

hub_stats_tool = FunctionTool.from_defaults(get_hub_stats)

print(hub_stats_tool("facebook")) # Example: Get the most downloaded model by Facebook

The most downloaded model by facebook is facebook/esmfold_v1 with 22,216,629 downloads.


## Integrating Tools to Alfred

In [14]:
from llama_index.core.agent.workflow import AgentWorkflow
from llama_index.llms.ollama import Ollama

llm = Ollama(
        model="llama3.2",
        temperature=0.1,
        request_timeout=120.0
    )
alfred = AgentWorkflow.from_tools_or_functions(
    [guest_info_tool, search_tool, weather_info_tool, hub_stats_tool],
    llm=llm
)

In [15]:
response = await alfred.run("What's the weather like in Paris?")
print(f"🌤️ Alfred's Weather Response: {response}")

🌤️ Alfred's Weather Response: The current weather in Paris is rainy with a temperature of 15°C (59°F). It's a good idea to carry an umbrella and wear waterproof clothing if you're planning to visit the city.


In [16]:
response = await alfred.run("What is Facebook and what's their most popular model?")

print("🎩 Alfred's Response:")
print(response)

🎩 Alfred's Response:
Facebook is a social media platform where users can create profiles, connect with friends and family, share updates and photos, and join groups based on common interests.

As for their most popular model, I couldn't find any specific information on a "model" associated with Facebook. However, the company has developed several products and technologies over the years, including:

1. News Feed: a feature that displays a curated feed of posts from friends and pages you follow.
2. Instagram: a photo and video-sharing platform acquired by Facebook in 2012.
3. WhatsApp: a messaging app acquired by Facebook in 2014.

These products have become incredibly popular among users, with billions of people using them every day.


In [17]:
query = "Tell me about Lady Ada Lovelace. What's her background?"
response = await alfred.run(query)

print("🎩 Alfred's Response:")
print(response.response.blocks[0].text)

🎩 Alfred's Response:
Lady Ada Lovelace (1815-1852) was a British mathematician and writer, often considered the world's first computer programmer. She was born Augusta Ada Byron, the daughter of the poet Lord George Gordon Byron and his wife Anne Isabella Milbanke.

Ada's early life was marked by tragedy when her mother died when she was just eight years old. Her father abandoned the family, leaving them in poverty. To support herself, Ada took up music lessons and later became proficient in mathematics, a subject she found fascinating.

In 1833, Ada met Augustus De Morgan, a mathematician who introduced her to Charles Babbage's proposed mechanical general-purpose computer, the Analytical Engine. This encounter sparked her interest in mathematics and computing, which would eventually become her life's work.

Ada collaborated with Babbage on his engine design, writing notes and specifications for its operation. Her work on the Analytical Engine is considered the first algorithm intended

In [18]:
query = "What's the weather like in Paris tonight? Will it be suitable for our fireworks display?"
response = await alfred.run(query)

print("🎩 Alfred's Response:")
print(response)

🎩 Alfred's Response:
The current weather conditions in Paris are mostly clear with a temperature of 20°C (68°F). However, it's also quite windy, which might affect the stability and visibility of your fireworks display.

Considering this, I would recommend taking necessary precautions to ensure the safety of your event. You may want to consider using wind-resistant fireworks or adjusting the timing of your display to minimize the impact of the wind.

Additionally, you can check the forecast for any potential changes in the weather before the event. The Meteo France website provides up-to-date weather forecasts and warnings for Paris.

Please let me know if there's anything else I can help with!


In [20]:
query = "One of our guests is from Google. What can you tell me about their most popular model?"
response = await alfred.run(query)

print("🎩 Alfred's Response:")
print(response)

🎩 Alfred's Response:
It seems that I couldn't find any information on a specific model from Google. However, I can tell you about some of the most popular machine learning models used by Google in various applications.

1. **BERT (Bidirectional Encoder Representations from Transformers)**: BERT is a pre-trained language model developed by Google that has achieved state-of-the-art results in many natural language processing tasks such as question answering, sentiment analysis, and text classification.
2. **TensorFlow**: TensorFlow is an open-source machine learning framework developed by Google that allows developers to easily implement and deploy machine learning models.
3. **Google Cloud AI Platform**: Google Cloud AI Platform is a fully managed platform for building, deploying, and managing machine learning models in the cloud.

These are just a few examples of popular models and technologies used by Google. If you have any specific questions or would like more information on these t

In [21]:
query = "I need to speak with Dr. Nikola Tesla about recent advancements in wireless energy. Can you help me prepare for this conversation?"
response = await alfred.run(query)

print("🎩 Alfred's Response:")
print(response)

🎩 Alfred's Response:
It seems I have some unexpected guests. Let me try again.

I've managed to arrange a meeting with Dr. Nikola Tesla regarding recent advancements in wireless energy. Before our conversation, let's review some key points about his work:

*   Wireless power transmission using electromagnetic induction
*   Patented a system for transmitting electrical energy wirelessly over long distances
*   Experimented with various frequencies and resonance techniques to optimize efficiency

Before meeting Dr. Tesla, you may want to consider the following topics to discuss during your conversation:

1.  **Wireless Power Transmission**: Ask about his experiences with wireless power transmission, including any challenges he faced and how he overcame them.
2.  **Applications of Wireless Energy**: Discuss potential applications for wireless energy, such as powering devices in remote areas or eliminating the need for cables.
3.  **Future Research Directions**: Inquire about Dr. Tesla's t

### Implementing memory

In [23]:
from llama_index.core.workflow import Context

# Remembering state
ctx = Context(alfred)

# First interaction
response1 = await alfred.run("Tell me about Lady Ada Lovelace.", ctx=ctx)
print("🎩 Alfred's First Response:")
print(response1)

# Second interaction (referencing the first)
response2 = await alfred.run("What projects is she currently working on?", ctx=ctx)
print("🎩 Alfred's Second Response:")
print(response2)

🎩 Alfred's First Response:
Lady Ada Lovelace (1815-1852) was a British mathematician and writer who is widely recognized as the world's first computer programmer. She is best known for her work on Charles Babbage's proposed mechanical general-purpose computer, the Analytical Engine.

Lovelace was born Augusta Ada King, Countess of Lovelace, to Lord George Gordon Byron and Anne Isabella Milbanke. Her father was a famous poet, and her mother encouraged her interest in mathematics and science from an early age. Lovelace's mathematical abilities were evident from a young age, and she went on to study mathematics at the University of Cambridge.

In 1843, Lovelace met Charles Babbage, who was working on his Analytical Engine project. Babbage showed her his designs for the machine, and Lovelace was fascinated by its potential. She wrote a set of notes on the engine, which included what is considered to be the first computer program.

Lovelace's notes on the Analytical Engine were based on an 